In [1]:
import sentencepiece as spm
import numpy as np
import tensorflow as tf
import pandas as pd
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#from tqdm import tqdm
from glob import glob


In [2]:
from glob import glob
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [3]:
model = tf.keras.models.load_model("classification_model.h5")

In [13]:
sp = spm.SentencePieceProcessor(model_file='full_30k_3m.model')

In [ ]:
# IMPORTANT define the location and the appropriate slices for the location.
directory_sorc = 'K:/US_applications/all_cleaned_modified/'
listoffiles = glob(directory_sorc+"*.csv")
print(listoffiles[0][40:-9])
print(len(listoffiles))
directory_sorc = 'K:/US_grants/all_cleaned_modified/'
listoffiles2 = glob(directory_sorc+"*.csv")
print(listoffiles2[-1][34:-9])
print(len(listoffiles2))
directory_sorc = 'K:/EU/EU_modified/'
listoffiles3 = glob(directory_sorc+"*.csv")
print(listoffiles3[0][18:-4])
print(len(listoffiles3))
listoffiles = listoffiles + listoffiles2  + listoffiles3

In [ ]:

start_num = 0

for num_fil,file in enumerate(listoffiles):
    #if num_fil < 11:
        #continue
    print(file)
    PAT = []
    YEAR = []
    TYP = []
    MOD = []
    prev = []
    collected = []
    prob = []
    nontype = False
    X = []
    if 'cleaned' in file[34:]:
        nontype = True
        data = pd.read_table(file, sep='\t',header=0,names=["PatentNumber",'PatentYear', "Paragraph","Modified"],converters = {'PatentNumber' : str,'PatentYear' : str,'Paragraph':str,'Modified':int})
    else:
        data = pd.read_table(file, sep='\t',header=0,names=["PatentNumber",'PatentYear','PatentType', "Paragraph","Modified"],converters = {'PatentNumber' : str,'PatentYear' : str,'Paragraph':str,'PatentType':str,'Modified':int})
    datap=data.copy(deep=True)
    data['totalwords'] = [len(x.split()) for x in data['Paragraph'].tolist()]
    
    data.dropna(subset=['Paragraph'],inplace=True)
    data = data[data['totalwords'] < 500]
    data = data[data['totalwords'] > 10]
    data = data[data['Paragraph']!='']
    del data['totalwords']
    total_len = len(data)
    print('total: ',len(data))
    
    data['TBR'] = 0

    data['Paragraph_en'] = data['Paragraph'].apply(lambda x: sp.encode(x))
    data['totalwords_en'] = [len(x) for x in data['Paragraph_en'].tolist()]
    data = data[data['totalwords_en'] < 30000]

    X = [np.array(fen,dtype=np.int32) for fen in data['Paragraph_en']]

    print('to_rag')
    X = tf.ragged.constant(X)
    
    y_pred = model.predict(X,verbose=1,batch_size=256)
    
    y_copy = y_pred.copy()
    
    y_pred[y_pred>=0.5] = 1
    y_pred[y_pred<1-0.5] = 0
    rez = []
    rezf = []
    for z in y_pred.tolist():
        rez.append(int(z[0]))
    for z in y_copy.tolist():
        rezf.append(round(float(z[0]),3))
    data['Predicted_2'] = rez
    data['Predicted'] = rezf
    data = data[data['Predicted_2']==1]
    data['Paragraph'] = data['Paragraph'].str.replace('\n','')
    del data['Predicted_2']
    #data.reset_index(drop=True,inplace=True)
    for z in list(data.index):
        start_num += 1
        #if start_num in to_take:
        if z != 0:
            prev.append(datap.at[z-1,'Paragraph'])
        
        collected.append(data.at[z,'Paragraph'])
        PAT.append(data.at[z,'PatentNumber'])
        YEAR.append(data.at[z,'PatentYear'])
        MOD.append(data.at[z,'Modified'])
        #MOD.append(data.at[z,'Modified'])
        prob.append(data.at[z,'Predicted'])
        if nontype == True:
            TYP.append('unknown')
        else:
            TYP.append(data.at[z,'PatentType'])
            
    dicte = {'PatentNumber': PAT, 'PatentYear': YEAR,'PatentType':TYP, 'Paragraph': collected,'Modified':MOD,'Predicted':prob,'PreviousP':prev}
    df = pd.DataFrame(dicte,columns =['PatentNumber','PatentYear','PatentType','Paragraph','PreviousP','Modified','Predicted'])
    if num_fil < 22:
        print('applic: ',start_num, round(start_num/total_len,2))
        df.to_csv('K:/Full_run/applic_'+listoffiles[num_fil][40:-9]+'.csv',sep='\t',index=False)
    if 21 < num_fil < 74:
        print('grants: ',start_num, round(start_num/total_len,2))
        df.to_csv('K:/Full_run/grants_'+listoffiles[num_fil][34:-9]+'.csv',sep='\t',index=False)
    if 73 < num_fil:
        print('EU: ',start_num, round(start_num/total_len,2))
        df.to_csv('K:/Full_run/EU_pat_'+listoffiles[num_fil][18:-4]+'.csv',sep='\t',index=False)